# Exercise - Decision Tree

This data set for this exercise is from healthcare. It contains data about 10 years of clinical care at 130 US Hospitals. Each row represents a single patient. The columns include the characteristics of deidentified diabetes patients. This is a binary classification task: predict whether a diabetes patient is readmitted to the hospital within 30 days of their discharge (1=Yes, 0=No). This is an important performance metric for hospitals as they try to minimize these types of readmissions.

## Description of Variables

The description of variables are provided in "Healthcare (small) - Data Dictionary.docx"

## Goal

Use the **healthcare_small.csv** data set and build a model to predict **readmitted**. 

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

diabetes = pd.read_csv("healthcare_small.csv")
diabetes.head()

,race,gender,age,admission_type,discharge_disposition,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
0,Other,Female,70-80,2,3,1,14,32,3,15,0,0,0,9,1
1,Caucasian,Female,80-90,1,3,5,4,44,0,15,0,0,0,9,0
2,AfricanAmerican,Male,50-60,5,1,1,6,29,1,15,0,0,0,9,1
3,Caucasian,Female,50-60,1,1,6,3,47,0,10,0,0,0,4,0
4,AfricanAmerican,Female,40-50,3,1,1,4,92,0,15,0,0,0,7,0


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, test_size=0.3)

# Data Prep

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [5]:
train_target = train['readmitted']
test_target = test['readmitted']

train_inputs = train.drop(['readmitted'], axis=1)
test_inputs = test.drop(['readmitted'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Whether the patient had any emergency visits or not
- Ratio of inpatient visits to outpatient visits
- Ratio of time in hospital to number of medications
- (etc.)

In [6]:
# Derived column: ratio of time_in_hospital per number_inpatient
def new_col(df):

    df1 = df.copy()
    df1['days_in_hosp_per_inpatient_visit'] = (df1['time_in_hospital']/df1['number_inpatient']).fillna(0)
    df1['days_in_hosp_per_inpatient_visit'].replace(np.inf, 1, inplace=True)
    return df1[['days_in_hosp_per_inpatient_visit']]


In [7]:
new_col(train)

,days_in_hosp_per_inpatient_visit
4166,1.000000
5546,1.000000
2957,1.000000
6329,1.000000
565,2.000000
...,...
5734,3.333333
5191,2.000000
5390,14.000000
860,1.000000


##  Identify the numeric, binary, and categorical columns

In [8]:
# Let's change some of the categorical variables' data type to object

train_inputs['admission_source'] = train_inputs['admission_source'].astype('object')
train_inputs['admission_type'] = train_inputs['admission_type'].astype('object')
train_inputs['discharge_disposition'] = train_inputs['discharge_disposition'].astype('object')

test_inputs['admission_source'] = test_inputs['admission_source'].astype('object')
test_inputs['admission_type'] = test_inputs['admission_type'].astype('object')
test_inputs['discharge_disposition'] = test_inputs['discharge_disposition'].astype('object')


In [9]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [10]:
numeric_columns

['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses']

In [11]:
categorical_columns

['race',
 'gender',
 'age',
 'admission_type',
 'discharge_disposition',
 'admission_source']

In [12]:
feat_eng_columns = ['time_in_hospital','number_inpatient']

# Pipeline

In [13]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [14]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [15]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col)),
                               ('scaler', StandardScaler())])



In [16]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('trans', my_new_column, feat_eng_columns)],   
        remainder='drop')



# Transform: fit_transform() for TRAIN

In [17]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<6066x65 sparse matrix of type '<class 'numpy.float64'>'
	with 90990 stored elements in Compressed Sparse Row format>

In [18]:
train_x.shape

(6066, 65)

In [19]:
train_x.toarray()

array([[-0.18928586, -0.24375736, -0.19945286, ...,  0.        ,
         0.        , -0.42498884],
       [ 1.12182808,  1.50683437, -0.78791641, ...,  0.        ,
         0.        , -0.42498884],
       [-0.84484283, -0.14078138,  0.38901069, ...,  0.        ,
         0.        , -0.42498884],
       ...,
       [ 3.08849899, -0.60417331, -0.78791641, ...,  0.        ,
         0.        ,  5.48345623],
       [-1.17262132, -0.6556613 ,  2.15440134, ...,  0.        ,
         0.        , -0.42498884],
       [ 0.13849262, -1.06756524, -0.78791641, ...,  1.        ,
         0.        ,  1.39299426]])

# Tranform: transform() for TEST

In [20]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<2600x65 sparse matrix of type '<class 'numpy.float64'>'
	with 38997 stored elements in Compressed Sparse Row format>

In [21]:
test_x.shape

(2600, 65)

In [22]:
test_x.toarray()

array([[ 0.46627111,  0.6315385 , -0.78791641, ...,  0.        ,
         0.        , -0.42498884],
       [ 0.13849262,  1.09493043, -0.78791641, ...,  0.        ,
         0.        , -0.42498884],
       [-0.51706435,  1.3523704 , -0.78791641, ...,  0.        ,
         0.        , -0.42498884],
       ...,
       [-0.18928586,  0.52856252, -0.78791641, ...,  0.        ,
         0.        , -0.42498884],
       [ 0.79404959,  0.78600248, -0.19945286, ...,  0.        ,
         0.        ,  0.71125059],
       [ 0.46627111,  0.52856252,  0.38901069, ...,  0.        ,
         0.        , -0.42498884]])

# Calculate the Baseline

In [23]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [24]:
from sklearn.metrics import accuracy_score
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.5313221233102539


In [25]:
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5365384615384615


# Train a DT model

In [26]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(max_depth=5)

tree_clf.fit(train_x, train_target)

DecisionTreeClassifier(max_depth=5)

### Calculate the accuracy

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
#Train accuracy:
train_y_pred = tree_clf.predict(train_x)
print(accuracy_score(train_target, train_y_pred))

0.6252884932410155


In [29]:
#Test accuracy:
test_y_pred = tree_clf.predict(test_x)
print(accuracy_score(test_target, test_y_pred))

# Better than baseline.  2 points difference -> no overfitting.

0.6038461538461538


In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_target, test_y_pred)

array([[946, 449],
       [581, 624]], dtype=int64)

# Train another DT model (with different parameters)

In [31]:
from sklearn.tree import DecisionTreeClassifier 
tree_clf = DecisionTreeClassifier(max_depth=5,min_samples_leaf=10,)
tree_clf.fit(train_x, train_target)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=10)

### Calculate the accuracy

In [32]:
#Train accuracy:
train_y_pred2 = tree_clf.predict(train_x)
print(accuracy_score(train_target, train_y_pred2))

0.6234751071546324


In [33]:
#Test accuracy:
test_y_pred2 = tree_clf.predict(test_x)
print(accuracy_score(test_target, test_y_pred2))

# Better than baseline.  Slightly worse than previous model.  2 points difference -> no overfitting.

0.6034615384615385


In [34]:
confusion_matrix(test_target, test_y_pred2)

array([[945, 450],
       [581, 624]], dtype=int64)

# Optional: try grid search

In [35]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {'max_depth': randint(low=5, high=30), 
              'min_samples_leaf': randint(low=5, high=30)}

tree_gs = RandomizedSearchCV(DecisionTreeClassifier(), param_grid, 
                             n_iter=30, cv=5, verbose=1,
                             scoring='accuracy',
                             return_train_score=True)
tree_gs.fit(train_x, train_target)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=30,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000224FC3FA350>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000224FC3FACE0>},
                   return_train_score=True, scoring='accuracy', verbose=1)

In [36]:
cvres = tree_gs.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.5837474585456022 {'max_depth': 9, 'min_samples_leaf': 26}
0.5548940568335075 {'max_depth': 20, 'min_samples_leaf': 13}
0.5768254671047182 {'max_depth': 14, 'min_samples_leaf': 28}
0.5705586514027742 {'max_depth': 22, 'min_samples_leaf': 21}
0.5527516973587889 {'max_depth': 22, 'min_samples_leaf': 11}
0.5952866461765798 {'max_depth': 8, 'min_samples_leaf': 9}
0.5730329448546838 {'max_depth': 27, 'min_samples_leaf': 20}
0.5989146954125475 {'max_depth': 7, 'min_samples_leaf': 22}
0.5755053368844655 {'max_depth': 27, 'min_samples_leaf': 25}
0.5702280755842459 {'max_depth': 18, 'min_samples_leaf': 19}
0.5501133383404115 {'max_depth': 21, 'min_samples_leaf': 5}
0.5761645870993941 {'max_depth': 29, 'min_samples_leaf': 25}
0.5784725061151093 {'max_depth': 21, 'min_samples_leaf': 27}
0.5665990756372141 {'max_depth': 29, 'min_samples_leaf': 15}
0.5745134736130144 {'max_depth': 21, 'min_samples_leaf': 18}
0.5736916518061472 {'max_depth': 24, 'min_samples_leaf': 23}
0.5552253117313671 {'max_dept

In [37]:
tree_gs.best_params_

{'max_depth': 6, 'min_samples_leaf': 24}

In [38]:
tree_gs.best_estimator_

DecisionTreeClassifier(max_depth=6, min_samples_leaf=24)

In [39]:
train_y_pred3 = tree_gs.best_estimator_.predict(train_x)
print(accuracy_score(train_target, train_y_pred3))

0.625618199802176


In [40]:
test_y_pred3 = tree_gs.best_estimator_.predict(test_x)
print(accuracy_score(test_target, test_y_pred3))

0.5980769230769231


In [41]:
confusion_matrix(test_target, test_y_pred3)

array([[1017,  378],
       [ 667,  538]], dtype=int64)

In [42]:
# This model is worse than the initial model (max_depth=5 only)
# More overfitting here (not too much, just more)
# This is not a better model